# Generador de Datasets Sinteticos con LLMs

Este notebook permite generar datasets estructurados utilizando modelos de lenguaje.

**Caracteristicas:**
- Define un esquema (campos y tipos de datos)
- Especifica el contexto/dominio de los datos
- Genera automaticamente la cantidad de registros que necesites
- Proporciona una interfaz con Gradio

**Modelos soportados:**
- OpenAI
- Anthropic
- Google
- Hugging Face

## 1. Instalacion de Dependencias

In [55]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q openai anthropic google-generativeai
!pip install -q datasets pandas tqdm

  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...


In [45]:
import os
import re
import json
import hashlib
import pandas as pd
import time
from tqdm.auto import tqdm
from typing import List, Dict, Optional
from google.colab import userdata
from huggingface_hub import login

  [WARN] Respuesta vacía o inválida, reintentando...


## 2. Configuracion de APIs y Modelos

In [46]:
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Claves cargadas desde Colab Secrets")
except Exception as e:
    print(e)

print(f"OpenAI: {'OK' if OPENAI_API_KEY else 'No configurada'}")
print(f"Anthropic: {'OK' if ANTHROPIC_API_KEY else 'No configurada'}")
print(f"Google: {'OK' if GOOGLE_API_KEY else 'No configurada'}")
print(f"HuggingFace: {'OK' if HF_TOKEN else 'No configurado'}")

SYSTEM_MESSAGE = "Eres un generador de datos sinteticos experto. Siempre generas JSON válido."

OPENAI_MODEL="gpt-4o-mini"
ANTHROPIC_MODEL="claude-sonnet-4-5-20250929"
GOOGLE_MODEL="gemini-2.0-flash-exp"
HF_MODEL="meta-llama/Llama-3.2-3B-Instruct"

Claves cargadas desde Colab Secrets
OpenAI: OK
Anthropic: OK
Google: OK
HuggingFace: OK


## 3. Clases de Generadores

In [47]:
class OpenAIGenerator:
    def __init__(self, model: str = OPENAI_MODEL):
        from openai import OpenAI
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": SYSTEM_MESSAGE},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content


class AnthropicGenerator:
    def __init__(self, model: str = ANTHROPIC_MODEL):
        from anthropic import Anthropic
        self.client = Anthropic(api_key=ANTHROPIC_API_KEY)
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            temperature=temperature,
            system=SYSTEM_MESSAGE,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.content[0].text


class GeminiGenerator:
    def __init__(self, model: str = GOOGLE_MODEL):
        import google.generativeai as genai
        genai.configure(api_key=GOOGLE_API_KEY)
        self.model = genai.GenerativeModel(
            model_name=model,
            system_instruction=SYSTEM_MESSAGE
        )

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.model.generate_content(
            prompt,
            generation_config={
                'temperature': temperature,
                'max_output_tokens': max_tokens,
            }
        )
        return response.text


class HuggingFaceGenerator:
    def __init__(self, model_name: str = HF_MODEL):
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
        import torch

        login(HF_TOKEN, add_to_git_credential=True)
        self.model_name = model_name
        print(f"Cargando modelo: {model_name}")

        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4"
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto"
        )
        print("Modelo cargado")

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        messages = [
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": prompt}
        ]
        inputs = self.tokenizer.apply_chat_template(
            messages,
            return_tensors="pt"
        ).to("cuda")

        outputs = self.model.generate(
            inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        response = self.tokenizer.decode(
            outputs[0][inputs.shape[1]:],
            skip_special_tokens=True
        )
        return response.strip()

## 4. Generador de Datasets Sinteticos

In [48]:
import json
import re
import pandas as pd
from tqdm.auto import tqdm
from typing import List, Dict

def build_prompt(schema: Dict, context: str, batch_size: int) -> str:
    """Construye el prompt para el LLM."""

    schema_str = json.dumps(schema, indent=2, ensure_ascii=False)
    prompt = f"""Genera EXACTAMENTE {batch_size} de registros de datos sinteticos en formato JSON.

CONTEXTO DEL DATASET: {context}
ESQUEMA: {schema_str}

Instrucciones importantes:
1. Genera EXACTAMENTE {batch_size} de registros.
2. Devuelve SOLO un array JSON válido (iniciando con [ y terminando con ])
3. No incluyas explicaciones, comentarios ni texto adicional
4. Respeta los tipos de datos del esquema
5. Genera datos realistas y variados según el contexto

Respuesta (solo JSON):"""

    return prompt

def _clean_json(response: str) -> List[Dict]:
    """Limpia y parsea la respuesta del LLM."""

    # Eliminar bloques de código markdown si existen
    if "```" in response:
        response = re.sub(r'```json|```', '', response).strip()

    # Buscar los corchetes de la lista JSON para aislar el contenido
    start = response.find('[')
    end = response.rfind(']')

    if start != -1 and end != -1:
        response = response[start:end+1]

    try:
        return json.loads(response)
    except:
        return []

def generate_dataset(
    generator,
    schema: Dict,
    context: str,
    num_records: int,
    batch_size: int = 10,
    temperature: float = 0.7
) -> pd.DataFrame:
    """
    Función principal para generar el dataset.

    Args:
        generator: Instancia del generador (OpenAI, Anthropic, etc.)
        schema: Diccionario con el esquema de datos
        context: Descripción del contexto
        num_records: Número total de registros a generar
        batch_size: Registros por petición
        temperature: Creatividad del modelo
    """
    generated_data = []
    pbar = tqdm(total=num_records, desc="Generando datos")

    while len(generated_data) < num_records:
        # Calcular cuántos registros faltan
        remaining = num_records - len(generated_data)
        current_batch_size = min(batch_size, remaining)

        prompt = build_prompt(schema, context, current_batch_size)

        try:
            # Llamada al generador
            response = generator.generate(prompt, temperature=temperature)
            records = _clean_json(response)

            if records:
                generated_data.extend(records)
                pbar.update(len(records))
            else:
                print("  [WARN] Respuesta vacía o inválida, reintentando...")
        except Exception as e:
            print(f"  [ERROR] {e}")

    pbar.close()
    # Recortar al número exacto solicitado por si se generaron de más
    generated_data = generated_data[:num_records]
    return pd.DataFrame(generated_data)

def save_dataset(df: pd.DataFrame, filename: str):
    """Guarda el DataFrame en disco."""
    df.to_csv(filename, index=False)
    print(f"Guardado: {filename}")

## 5. Ejemplos de Uso

### Crear generador

In [49]:
# Elige el generador segun la API que tengas configurada:

generator = OpenAIGenerator("gpt-4o-mini")
# generator = AnthropicGenerator("claude-4-5-sonnet-latest")
# generator = GeminiGenerator("gemini-2.0-flash-exp")
# generator = HuggingFaceGenerator("meta-llama/Llama-3.2-3B-Instruct")
# generator = HuggingFaceGenerator("google/gemma-2-2b-it")

### Ejemplo 1: Dataset de Ventas

In [50]:
schema_ventas = {
    "producto": "nombre del producto (string)",
    "categoria": "categoria del producto (string)",
    "precio": "precio en euros (numero decimal)",
    "cantidad_vendida": "unidades vendidas (entero)",
    "fecha_venta": "fecha en formato YYYY-MM-DD",
    "ciudad": "ciudad de Espana donde se realizo la venta (string)"
}

df_ventas = generate_dataset(
    generator=generator,
    schema=schema_ventas,
    context="Ventas de una tienda online de electronica en Espana durante 2024",
    num_records=10,
    batch_size=5,
    temperature=0.8
)

display(df_ventas.head(10))

Generando datos:   0%|          | 0/10 [00:00<?, ?it/s]

  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...


,producto,categoria,precio,cantidad_vendida,fecha_venta,ciudad
0,Smartphone X100,Móviles,499.99,50,2024-01-15,Madrid
1,Auriculares Bluetooth Pro,Accesorios,89.99,150,2024-02-20,Barcelona
2,"Televisor 55"" 4K UHD",Televisores,799.99,30,2024-03-10,Valencia
3,Portátil Gamer Z500,Ordenadores,1299.99,20,2024-04-05,Sevilla
4,"Tableta 10"" HD",Tabletas,249.99,75,2024-05-12,Bilbao
5,Smartphone XYZ 2024,Teléfonos móviles,799.99,15,2024-02-15,Madrid
6,Auriculares Inalámbricos ABC,Accesorios,149.99,30,2024-03-05,Barcelona
7,"Televisor 55"" Ultra HD",Televisores,999.00,7,2024-01-20,Valencia
8,Ordenador Portátil Gamer Pro,Ordenadores,1299.99,5,2024-04-10,Sevilla
9,"Tableta 10"" con Estilógrafo",Tabletas,349.99,20,2024-05-25,Bilbao


### Ejemplo 2: Dataset de Paises

In [51]:
schema_paises = {
    "pais": "nombre del pais",
    "capital": "nombre de la capital",
    "continente": "continente donde se encuentra",
    "poblacion_millones": "poblacion aproximada en millones (numero)",
    "idioma_oficial": "idioma oficial principal",
    "moneda": "nombre de la moneda oficial"
}

df_paises = generate_dataset(
    generator=generator,
    schema=schema_paises,
    context="Paises del mundo con datos geograficos y demograficos",
    num_records=10,
    batch_size=5,
    temperature=0.5
)

display(df_paises.head(10))

Generando datos:   0%|          | 0/10 [00:00<?, ?it/s]

  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...
  [WARN] Respuesta vacía o inválida, reintentando...


,pais,capital,continente,poblacion_millones,idioma_oficial,moneda
0,Argentina,Buenos Aires,América del Sur,45.38,Español,Peso argentino
1,Japón,Tokio,Asia,126.85,Japonés,Yen japonés
2,Alemania,Berlín,Europa,83.24,Alemán,Euro
3,Nigeria,Abuja,África,211.40,Inglés,Naira nigeriano
4,Australia,Canberra,Oceanía,25.69,Inglés,Dólar australiano
5,Brasil,Brasilia,América del Sur,213.00,Portugués,Real brasileño
6,Japón,Tokio,Asia,126.00,Japonés,Yen japonés
7,Alemania,Berlín,Europa,83.00,Alemán,Euro
8,Sudáfrica,Pretoria,África,60.00,Afrikáans,Rand sudafricano
9,Australia,Canberra,Oceanía,25.00,Inglés,Dólar australiano


### Guardar Datasets

In [52]:
# Guardar en CSV
save_dataset(df_ventas, 'dataset_ventas.csv')
save_dataset(df_paises, 'dataset_paises.csv')

print("Datasets guardados")

Guardado: dataset_ventas.csv
Guardado: dataset_paises.csv
Datasets guardados


## 6. Interfaz Gradio

In [ ]:
import gradio as gr
import json
import pandas as pd

# Elige el generador segun la API que tengas configurada:

generator = OpenAIGenerator("gpt-4o-mini")
# generator = AnthropicGenerator("claude-4-5-sonnet-latest")
# generator = GeminiGenerator("gemini-2.0-flash-exp")
# generator = HuggingFaceGenerator("meta-llama/Llama-3.2-3B-Instruct")
# generator = HuggingFaceGenerator("google/gemma-2-2b-it")

def gradio_generate(context, schema_str, num_records, temperature):
    try:
        # Parsear el esquema desde el string JSON
        schema = json.loads(schema_str)
    except json.JSONDecodeError:
        return pd.DataFrame({'Error': ["El esquema no es un JSON válido."]}), None

    try:
        # Generar el dataset usando la función existente
        # Asume que 'generator' ya está instanciado en celdas anteriores
        df = generate_dataset(
            generator=generator,
            schema=schema,
            context=context,
            num_records=int(num_records),
            batch_size=min(10, int(num_records)),
            temperature=float(temperature)
        )

        # Guardar temporalmente para descarga
        output_filename = "dataset_generado.csv"
        save_dataset(df, output_filename)

        return df, output_filename
    except Exception as e:
        return pd.DataFrame({'Error': [str(e)]}), None

# Esquema por defecto
default_schema = """{
    "producto": "nombre del producto (electronica, ropa, hogar)",
    "categoria": "categoria del producto",
    "precio": "precio en euros (numero decimal)",
    "cantidad_vendida": "unidades vendidas (entero)",
    "fecha_venta": "fecha en formato YYYY-MM-DD",
    "ciudad": "ciudad de Espana donde se realizo la venta"
}"""

# Definición de la interfaz
with gr.Blocks(title="Generador de Datasets Sintéticos") as demo:
    gr.Markdown("# Generador de Datasets Sintéticos")
    gr.Markdown("Define el contexto y el esquema para generar datos estructurados usando LLMs.")

    with gr.Row():
        with gr.Column():
            context_input = gr.Textbox(
                label="Contexto del Dataset",
                placeholder="Ej: Datos de ventas de productos de una tienda online...",
                lines=3
            )
            schema_input = gr.Code(
                label="Esquema (JSON)",
                value=default_schema,
                language="json"
            )
            with gr.Row():
                num_records = gr.Number(label="Número de registros", value=10, precision=0)
                temp_slider = gr.Slider(label="Temperatura", minimum=0.0, maximum=1.0, value=0.7)

            generate_btn = gr.Button("Generar Dataset", variant="primary")

        with gr.Column():
            output_df = gr.Dataframe(label="Vista Previa")
            download_file = gr.File(label="Descargar CSV")

    generate_btn.click(
        fn=gradio_generate,
        inputs=[context_input, schema_input, num_records, temp_slider],
        outputs=[output_df, download_file]
    )

demo.launch(share=True, debug=True)
